In [1]:
import torch
import os
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm
from training_data import CombinedData
from PIL import Image
from matplotlib import pyplot as plt

data_train = CombinedData('HASY')
data_test = CombinedData('HASY', train=False)

print("Train data length: {0}".format(len(data_train.data)))
print("Test data length: {0}".format(len(data_test.data)))
print("Img Shape: {0}".format(data_train.data[0].shape))
print("Number of Labels: {0}".format(data_train.no_labels))

100%|██████████| 10000/10000 [00:00<00:00, 367399.31it/s]

Train data length: 61958
Test data length: 10224
Img Shape: torch.Size([1, 28, 28])
Number of Labels: 15


In [ ]:
# Train with a pretrained model from PyTorch
from torchvision import models
from torch.nn import Conv2d

pretrained = False
torch_model = models.densenet201(num_classes=15)
torch_model.features.conv0 = Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
if pretrained:
    torch_model.load_state_dict(torch.load('pretrained-model-01.ckpt'))
epochs = 10
optimizer = torch.optim.Adam(torch_model.parameters(), lr=0.001, betas=(0.8, 0.925), weight_decay=0.01)
criterion = nn.CrossEntropyLoss()

def calc_accuracy(model):
    accuracies = []
    for idx, [x_test, y_test] in enumerate(tqdm(test_loader)):
        test_pred = model(x_test)
        accuracy = 100 * torch.mean((torch.argmax(test_pred, dim=1) == y_test).float())
        accuracies.append(accuracy)
    print("Accuracy: {0}".format(np.mean(accuracies)))  

train_loader = DataLoader(data_train, batch_size=16, shuffle=True)
test_loader = DataLoader(data_test, batch_size=16, shuffle=False)

for epoch in range(epochs):
        print("Epoch {0}".format(epoch))
        calc_accuracy(torch_model)
        for step, [x_train, y_train] in enumerate(tqdm(train_loader)):
            optimizer.zero_grad()
            train_pred = torch_model(x_train)
            loss = criterion(train_pred, y_train)
            loss.backward()
            optimizer.step()
            if step % 100 == 0:
                print('Loss: {}'.format(loss))
torch.save(torch_model.state_dict(), 'combined-model.ckpt')

  0%|          | 1/639 [00:00<02:00,  5.30it/s]

Epoch 0


  0%|          | 0/3873 [00:00<?, ?it/s]

Accuracy: 5.3012518882751465


  0%|          | 1/3873 [00:01<1:12:55,  1.13s/it]

...

 57%|█████▋    | 2201/3873 [38:01<26:42,  1.04it/s]

Loss: 0.12622679769992828


 59%|█████▉    | 2301/3873 [39:47<28:06,  1.07s/it]

Loss: 0.458736777305603


 62%|██████▏   | 2383/3873 [41:08<24:45,  1.00it/s]

In [ ]:
print(data_train.data)